In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os

In [10]:
DATASET_DIR = 'output_data_self_norm_16k_1024'
SAVE_MODEL_NAME = "vibration_classifier_model_1024_16k_NORM_e5.keras"
BATCH_SIZE = 16
IMG_HEIGHT = 513
IMG_WIDTH = 15361
EPOCHS = 4
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
print("Загрузка обучающего набора данных")
train_dataset = tf.keras.utils.image_dataset_from_directory(
    f'{DATASET_DIR}/train',
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

print("\nЗагрузка валидационного набора данных")
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    f'{DATASET_DIR}/validation',
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

class_names = train_dataset.class_names
print(f"\nНайденные классы: {class_names}")

Загрузка обучающего набора данных
Found 72 files belonging to 2 classes.

Загрузка валидационного набора данных
Found 18 files belonging to 2 classes.

Найденные классы: ['defect', 'normal']


In [4]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [5]:
print("\nСоздание архитектуры сверточной нейронной сети (CNN)...")

model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    
    layers.Conv2D(16, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Dropout(0.2),

    layers.Flatten(), 
    layers.Dense(128, activation='relu'), 
    layers.Dense(1)
])


Создание архитектуры сверточной нейронной сети (CNN)...


In [6]:
print("Компиляция модели...")
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.summary()

Компиляция модели...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 513, 15361, 3)     0         
                                                                 
 conv2d (Conv2D)             (None, 513, 15361, 16)    448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 7680, 16)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 7680, 32)     4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 128, 3840, 32)    0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None,

In [7]:

print("\nНачинаем обучение модели...")

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS
)


Начинаем обучение модели...
Epoch 1/4
5/5 [==============================] - 74s 16s/step - loss: 19.3718 - accuracy: 0.5972 - val_loss: 1.4820e-04 - val_accuracy: 1.0000
Epoch 2/4
5/5 [==============================] - 70s 14s/step - loss: 0.2107 - accuracy: 0.9167 - val_loss: 0.0241 - val_accuracy: 1.0000
Epoch 3/4
5/5 [==============================] - 73s 16s/step - loss: 0.1474 - accuracy: 0.9722 - val_loss: 0.1086 - val_accuracy: 1.0000
Epoch 4/4
5/5 [==============================] - 82s 17s/step - loss: 0.0687 - accuracy: 1.0000 - val_loss: 0.0302 - val_accuracy: 1.0000


In [11]:
print(f"\nСохранение обученной модели в файл: {SAVE_MODEL_NAME}")
model.save(SAVE_MODEL_NAME)
print("Модель успешно сохранена.")


Сохранение обученной модели в файл: vibration_classifier_model_1024_16k_NORM_e5.keras
Модель успешно сохранена.
